# Lesson 4: Streamlining User Interaction with Chat Controller

Welcome to the next step in our journey of building a chatbot service. In the previous lesson, we explored the `ChatService` class, which acts as a bridge between managing chat data and generating AI responses. Now, we will focus on the `ChatController`, a crucial component that manages chat sessions and handles messages by interacting with both the model and service layers. The controller is responsible for orchestrating the flow of data between the user interface and the backend services, ensuring that user interactions are processed efficiently and effectively.

## Implementing the ChatController Class

The `ChatController` class is the heart of our controller layer. It is responsible for managing chat sessions and processing user messages. Let's begin by examining the structure of the `ChatController` class.

```python
import uuid
from services.chat_service import ChatService

class ChatController:
    def __init__(self):
        self.chat_service = ChatService()
        self.test_session = {}  # Simple session storage for testing
```

In this snippet, we:
- Import the `uuid` module from the Python standard library for generating unique identifiers.
- Import the `ChatService` class for managing chat data and processing messages.
- Initialize the `ChatController` with an instance of `ChatService`.
- Create a `test_session` dictionary to simulate session management for testing purposes.

The `test_session` is a simple dictionary used to simulate session management for testing purposes. It allows us to mimic the behavior of user sessions typically managed by a web application or browser. In a real-world scenario, user sessions help track individual users as they interact with a web application, maintaining their state and data across multiple requests. By using `test_session`, we can focus on testing the core functionality of the `ChatController` without needing a full session management system. Once we are confident that the controller works correctly, we will later integrate a more robust session management solution when developing our RESTful API.

## Ensuring User Session

Before creating a chat, we need to ensure that a user session exists. The `ensure_user_session` method checks if a user ID is present in the `test_session`. If not, it generates a new user ID.

```python
def ensure_user_session(self):
    """Ensure user has a session ID in the test session."""
    if 'user_id' not in self.test_session:
        self.test_session['user_id'] = str(uuid.uuid4())
    return self.test_session['user_id']
```

This method ensures that a user session is available by checking the `test_session` dictionary for a `user_id`. If it doesn't exist, a new user ID is generated using `uuid.uuid4()` and stored in the session. The method then returns the user ID, either the newly created one or the existing one.

## Creating a New Chat Session

One of the primary responsibilities of the `ChatController` is to create new chat sessions. The `create_chat` method handles chat creation requests.

```python
def create_chat(self):
    """Handle chat creation request."""
    user_id = self.test_session.get('user_id')
    if not user_id:
        return {'error': 'Session expired'}, 401
    
    chat_id = self.chat_service.create_chat(user_id)
    return {
        'chat_id': chat_id,
        'message': 'Chat created successfully'
    }
```

In this method:
- **Retrieve the user_id:** We first check the `test_session` for a `user_id`.
- **Handle Session Expiry:** If the session has expired (i.e., no `user_id` is found), we return an error response: `{'error': 'Session expired'}, 401`. This approach ensures that the client can handle the error gracefully, providing a better user experience.
- **Create a Chat Session:** If the session is valid, we call the `create_chat` method of the `ChatService` with the user ID to create a new chat session. We then return a response containing a unique chat ID and a success message: `{'chat_id': chat_id, 'message': 'Chat created successfully'}`.

## Handling User Messages

The `send_message` method is responsible for processing user messages and returning the AI's response or an error message.

```python
def send_message(self, chat_id, user_message):
    """Handle message sending request."""
    user_id = self.test_session.get('user_id')
    if not user_id:
        return {'error': 'Session expired'}, 401
    
    if not chat_id or not user_message:
        return {'error': 'Missing chat_id or message'}, 400
        
    try:
        ai_response = self.chat_service.process_message(user_id, chat_id, user_message)
        return {'message': ai_response}
    except ValueError as e:
        return {'error': str(e)}, 404
    except RuntimeError as e:
        return {'error': str(e)}, 500
```

In this method:
- We first check if the user session is valid by retrieving the `user_id` from the `test_session`. If no `user_id` is found, we return an error response with a 401 status code.
- We then validate that both `chat_id` and `user_message` are provided. If either is missing, we return an error response with a 400 status code.
- If all validations pass, we attempt to process the message using the `process_message` method of the `ChatService`. This method takes the `user_id`, `chat_id`, and `user_message` as parameters.
- If the message is processed successfully, we return the AI's response in a dictionary: `{'message': ai_response}`.
- If a `ValueError` occurs (e.g., if the chat ID doesn't exist), we return an error response with a 404 status code.
- If a `RuntimeError` occurs (e.g., if there's an issue with the AI model), we return an error response with a 500 status code.

## Integrating the Chat Controller in the Main Application

To see the `ChatController` in action, let's integrate it into the main application. This example demonstrates how to create a chat session and handle a user message, showcasing the controller's functionality.

```python
from controllers.chat_controller import ChatController

# Initialize the ChatController
chat_controller = ChatController()

# Ensure a user session for testing
user_id = chat_controller.ensure_user_session()

# Create a new chat session
chat_response = chat_controller.create_chat()

# Handle chat creation response
if 'error' in chat_response:
    print(f"Error: {chat_response['error']}")
else:
    # Extract chat_id from the response and print the chat session details
    chat_id = chat_response['chat_id']
    print(f"Chat session created with chat_id: {chat_id}")

    # Example message handling
    user_message = "What are the support hours for the Basic Plan?"

    # Send the user message and get the response
    response = chat_controller.send_message(chat_id, user_message)

    # Handle message response
    if 'error' in response:
        print(f"Error: {response['error']}")
    else:
        print(f"AI Response: {response['message']}")
```

In this example, we first initialize the `ChatController`. We ensure a user session is available for testing. We then create a new chat session and handle the response. If successful, we simulate a user message and use the `send_message` method to process it. The response is checked for errors, and either the error message or the AI's response is printed. This example demonstrates the flow from ensuring a user session to creating a chat session and handling a user message, highlighting the controller's role in managing interactions.

## Summary and Next Steps

In this lesson, we explored the `ChatController` class and its role in managing chat sessions and handling user messages. We learned how to implement the controller, create chat sessions, and process messages using the `ChatService`. The controller is a vital component of our chatbot application, ensuring that user interactions are managed efficiently and effectively.

As you move on to the practice exercises, take the opportunity to experiment with the `ChatController`'s functionality. This hands-on practice will reinforce the concepts covered in this lesson and prepare you for the next steps in our course. Keep up the great work, and I look forward to seeing your progress!


## Ensuring Unique User Sessions

Great job on understanding the ChatController class so far! Now, let's put that knowledge into practice by focusing on the ensure_user_session method. This method is crucial for managing user sessions by ensuring each user has a unique session ID.

Your task is to complete the missing parts of the code:

Define the ensure_user_session method in the ChatController class:

Check if the test_session dictionary contains a key 'user_id'.
If 'user_id' is not present, generate a new unique user ID using the uuid module and store it in the test_session dictionary.
Return the user_id from the test_session.
Call this method in the main application to verify its functionality:

Initialize an instance of ChatController.
Use the ensure_user_session method to ensure a user session is created.
Print the user_id to verify that the session has been successfully created.
Dive in and see how this method helps in managing user sessions effectively!


```python
import uuid


class ChatController:
    def __init__(self):
        self.test_session = {}

    # TODO: Define the ensure_user_session method
    # - Check if 'user_id' is in the test_session dictionary
    # - Generate and store a new user_id using uuid if it doesn't exist
    # - Return the user_id from the test_session


```

```python
from controllers.chat_controller import ChatController

# TODO: Initialize the ChatController

# TODO: Ensure a user session for testing and print the user ID


```

Here's the implementation for ensuring unique user sessions:

First, let's complete the `ChatController` class with the `ensure_user_session` method:

```python
import uuid


class ChatController:
    def __init__(self):
        self.test_session = {}

    def ensure_user_session(self):
        """
        Ensures a user session exists by checking for a user_id in the session.
        If not present, generates a new unique user ID.

        Returns:
            str: The user ID from the session
        """
        if 'user_id' not in self.test_session:
            # Generate a new unique user ID
            self.test_session['user_id'] = str(uuid.uuid4())

        return self.test_session['user_id']
```

Now, let's implement the main application code to test this functionality:

```python
from controllers.chat_controller import ChatController

# Initialize the ChatController
chat_controller = ChatController()

# Ensure a user session for testing
user_id = chat_controller.ensure_user_session()
print(f"User session created with ID: {user_id}")

# Test that calling the method again returns the same ID
second_call_id = chat_controller.ensure_user_session()
print(f"Second call to ensure_user_session: {second_call_id}")
print(f"IDs match: {user_id == second_call_id}")

# Reset the session and verify a new ID is generated
chat_controller.test_session = {}
new_user_id = chat_controller.ensure_user_session()
print(f"New user session created with ID: {new_user_id}")
print(f"New ID is different from original: {user_id != new_user_id}")
```
This implementation:

1. Creates the `ensure_user_session` method that checks if a user ID exists in the session
2. Generates a new UUID if no user ID is found and stores it in the session
3. Returns the user ID from the session
4. Tests the method by initializing a controller and calling the method
5. Verifies that calling the method twice returns the same ID (session persistence)
6. Tests that resetting the session and calling the method again generates a new ID

The method ensures that each user has a unique identifier that persists throughout their session, which is essential for tracking conversations and maintaining context.

## Creating a New Chat Session with ChatController

## Implementing the Send Message Method in ChatController